In [ ]:
!/opt/bin/nvidia-smi

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
!git clone https://github.com/Paleuna/CS639.git
!git clone https://github.com/DrSleep/DenseTorch.git
!pip install -e /content/DenseTorch

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd /content/DenseTorch

In [ ]:
import densetorch

In [ ]:
# below is the linux command to download the required dataset
# we already downloaded the dataset to our google drive
# the command is for reference

In [ ]:
cd /content/gdrive/MyDrive/CS639/data1/Cityscapes/

/content/gdrive/MyDrive/CS639/data1/Cityscapes


In [ ]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=Paleuna&password=Abc123456!&submit=Login' "https://www.cityscapes-dataset.com/login/"
!wget --load-cookies cookies.txt --content-disposition "https://www.cityscapes-dataset.com/file-handling/?packageID=1"
!wget --load-cookies cookies.txt --content-disposition "https://www.cityscapes-dataset.com/file-handling/?packageID=3"
!unzip gtFine_trainvaltest.zip 
!rm gtFine_trainvaltest.zip 
!unzip leftImg8bit_trainvaltest.zip
!rm leftImg8bit_trainvaltest.zip

In [ ]:
cd /content/gdrive/MyDrive/CS639/data1/

In [ ]:
!wget --load-cookies cookies.txt --content-disposition "https://www.cityscapes-dataset.com/file-handling/?packageID=31"
!unzip leftImg8bit_trainval_foggyDBF.zip
!rm leftImg8bit_trainval_foggyDBF.zip

!wget "https://data.vision.ee.ethz.ch/csakarid/shared/Model_adaptation_SFSU_dense/Downloads/Foggy_Zurich.zip"
!unzip Foggy_Zurich.zip
!rm Foggy_Zurich.zip

!wget "https://data.vision.ee.ethz.ch/csakarid/shared/SFSU_synthetic/Downloads/Foggy_Driving.zip"
!unzip Foggy_Driving.zip
!rm Foggy_Driving.zip

In [ ]:
#here is the main code:

In [ ]:
cd /content/CS639/fifo

/content/CS639/fifo


In [ ]:
import os
import os.path as osp

import torch
import torch.nn as nn
from torch.utils import data
from torch.autograd import Variable
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.autograd import grad 

import numpy as np
import random
from tqdm import tqdm
from PIL import Image
from packaging import version
import argparse
from datetime import datetime

from model.refinenetlw import rf_lw101
from model.fogpassfilter import FogPassFilter_conv1, FogPassFilter_res1
from utils.losses import CrossEntropy2d
from dataset.paired_cityscapes import Pairedcityscapes
from dataset.Foggy_Zurich import foggyzurichDataSet
from utils.optimisers import get_optimisers, get_lr_schedulers
from pytorch_metric_learning import losses
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.reducers import MeanReducer

In [ ]:
#config of the training process

IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
BETA = 0.005
BATCH_SIZE = 4
ITER_SIZE = 1
NUM_WORKERS = 4
DATA_DIRECTORY ='/content/gdrive/MyDrive/CS639/data1'
DATA_LIST_PATH = f'./dataset/cityscapes_list/train_foggy_{BETA}.txt'
DATA_CITY_PATH = './dataset/cityscapes_list/clear_lindau.txt'
INPUT_SIZE = '2048,1024'
DATA_DIRECTORY_CWSF = '/content/gdrive/MyDrive/CS639/data1/Cityscapes'
DATA_LIST_PATH_CWSF = './dataset/cityscapes_list/train_origin.txt'
DATA_LIST_RF = '/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich/lists_file_names/RGB_sum_filenames.txt'
DATA_DIR = '/content/gdrive/MyDrive/CS639/data1'
INPUT_SIZE_RF = '1920,1080'
NUM_CLASSES = 19 
NUM_STEPS = 20000
NUM_STEPS_STOP = 13000  # early stopping
RANDOM_SEED = 1234
RESTORE_FROM = '/content/gdrive/MyDrive/CS639/Cityscapes_pretrained_model.pth'
RESTORE_FROM_fogpass = '/content/gdrive/MyDrive/CS639/FogPassFilter_pretrained.pth'
SAVE_PRED_EVERY = 100
SNAPSHOT_DIR = f'/content/gdrive/MyDrive/CS639/data1/snapshots/Model' 
FILE_NAME = 'Original_Model'  
SET = 'train'

def get_arguments():
    
    parser = argparse.ArgumentParser(description="MyModel")
    parser.add_argument("--img-mean", type=float, default=IMG_MEAN)
    parser.add_argument("--batch-size", type=int, default=BATCH_SIZE)
    parser.add_argument("--iter-size", type=int, default=ITER_SIZE)
    parser.add_argument("--num-workers", type=int, default=NUM_WORKERS)
    parser.add_argument("--data-dir", type=str, default=DATA_DIRECTORY)
    parser.add_argument("--data-list", type=str, default=DATA_LIST_PATH)
    parser.add_argument("--data-city-list", type=str, default = DATA_CITY_PATH)
    parser.add_argument("--data-list-rf", type=str, default=DATA_LIST_RF)    
    parser.add_argument("--input-size", type=str, default=INPUT_SIZE)
    parser.add_argument("--input-size-rf", type=str, default=INPUT_SIZE_RF)
    parser.add_argument("--data-dir-cwsf", type=str, default=DATA_DIRECTORY_CWSF)
    parser.add_argument("--data-list-cwsf", type=str, default=DATA_LIST_PATH_CWSF)
    parser.add_argument("--data-dir-rf", type=str, default=DATA_DIR)
    parser.add_argument("--num-classes", type=int, default=NUM_CLASSES)
    parser.add_argument("--num-steps", type=int, default=NUM_STEPS)
    parser.add_argument("--num-steps-stop", type=int, default=NUM_STEPS_STOP)
    parser.add_argument("--random-seed", type=int, default=RANDOM_SEED)
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM)
    parser.add_argument("--restore-from-fogpass", type=str, default=RESTORE_FROM_fogpass)
    parser.add_argument("--save-pred-every", type=int, default=SAVE_PRED_EVERY)
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR)
    parser.add_argument("--gpu", type=int, default=0)
    parser.add_argument("--set", type=str, default=SET)
    parser.add_argument("--lambda-fsm", type=float, default=0.0000001)
    parser.add_argument("--lambda-con", type=float, default=0.0001)
    parser.add_argument("--file-name", type=str, default = FILE_NAME)
    parser.add_argument("--modeltrain", type=str, default = 'train')
    return parser.parse_args(args=[])

args = get_arguments()


In [ ]:
def loss_calc(pred, label, gpu):
    label = Variable(label.long()).cuda(gpu)
    criterion = CrossEntropy2d().cuda(gpu)
    return criterion(pred, label)

def gram_matrix(tensor):
    d, h, w = tensor.size()
    tensor = tensor.view(d, h*w)
    gram = torch.mm(tensor, tensor.t())
    return gram

def make_list(x):
    """Returns the given input as a list."""
    if isinstance(x, list):
        return x
    elif isinstance(x, tuple):
        return list(x)
    else:
        return [x]

#get argument from the config and update the random seed
args = get_arguments()
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)
torch.cuda.manual_seed_all(args.random_seed)
torch.cuda.manual_seed(args.random_seed)

w, h = map(int, args.input_size.split(','))
w_r, h_r = map(int, args.input_size_rf.split(',')) 

input_size = (w, h)
input_size_rf = (w_r, h_r)   

#load the pretrain model for the segmentation network
restore = torch.load(args.restore_from)
model = rf_lw101(num_classes=args.num_classes)
model.load_state_dict(restore['state_dict'])
start_iter = 0

#gpu stuff
cudnn.enabled = True
gpu = args.gpu
model.train()
model.cuda(args.gpu)

lr_fpf1 = 1e-3 
lr_fpf2 = 1e-3

if args.modeltrain=='train':
    lr_fpf1 = 5e-4

#define the fog pass filter
FogPassFilter1 = FogPassFilter_conv1(2080)
FogPassFilter1_optimizer = torch.optim.Adamax([p for p in FogPassFilter1.parameters() if p.requires_grad == True], lr=lr_fpf1)
FogPassFilter1.cuda(args.gpu)
FogPassFilter2 = FogPassFilter_res1(32896)
FogPassFilter2_optimizer = torch.optim.Adamax([p for p in FogPassFilter2.parameters() if p.requires_grad == True], lr=lr_fpf2)
FogPassFilter2.cuda(args.gpu)

#load the pretrained model for the two filter
restore = torch.load(args.restore_from_fogpass)
FogPassFilter1.load_state_dict(restore['fogpass1_state_dict'])
FogPassFilter2.load_state_dict(restore['fogpass2_state_dict'])


fogpassfilter_loss = losses.ContrastiveLoss(
    pos_margin=0.1,
    neg_margin=0.1,
    distance=CosineSimilarity(),
    reducer=MeanReducer()
)

cudnn.benchmark = True

if not os.path.exists(args.snapshot_dir):
    os.makedirs(args.snapshot_dir)

max_iters = args.num_steps * args.iter_size * args.batch_size

#load the dataset (four datasets)
cwsf_pair_loader = data.DataLoader(
    Pairedcityscapes(args.data_dir, args.data_dir_cwsf, args.data_list, args.data_list_cwsf, max_iters=max_iters, mean=IMG_MEAN, set=args.set), 
    batch_size=args.batch_size, 
    shuffle=True, 
    num_workers=args.num_workers,
    pin_memory=True
)

rf_loader = data.DataLoader(
    foggyzurichDataSet(args.data_dir_rf, args.data_list_rf, max_iters=max_iters,mean=IMG_MEAN, set=args.set),
    batch_size=args.batch_size, 
    shuffle=True, 
    num_workers=args.num_workers,
    pin_memory=True
)

cwsf_pair_loader_fogpass = data.DataLoader(
    Pairedcityscapes(args.data_dir, args.data_dir_cwsf, args.data_list, args.data_list_cwsf,max_iters=max_iters,mean=IMG_MEAN, set=args.set), 
    batch_size=args.batch_size, 
    shuffle=True, 
    num_workers=args.num_workers,
    pin_memory=True
)

rf_loader_fogpass = data.DataLoader(
    foggyzurichDataSet(args.data_dir_rf, args.data_list_rf,max_iters=max_iters,mean=IMG_MEAN, set=args.set), 
    batch_size=args.batch_size, 
    shuffle=True, 
    num_workers=args.num_workers,
    pin_memory=True
)

#make the data be a interator
rf_loader_iter = enumerate(rf_loader)
cwsf_pair_loader_iter = enumerate(cwsf_pair_loader)
cwsf_pair_loader_iter_fogpass = enumerate(cwsf_pair_loader_fogpass)
rf_loader_iter_fogpass = enumerate(rf_loader_fogpass)

#optimization setting
optimisers = get_optimisers(
    model=model,
    enc_optim_type="sgd",
    enc_lr=6e-4,
    enc_weight_decay=1e-5,
    enc_momentum=0.9,
    dec_optim_type="sgd",
    dec_lr=6e-3,
    dec_weight_decay=1e-5,
    dec_momentum=0.9,
)

#learning setting
schedulers = get_lr_schedulers(
    enc_optim=optimisers[0],
    dec_optim=optimisers[1],
    enc_lr_gamma=0.5,
    dec_lr_gamma=0.5,
    enc_scheduler_type="multistep",
    dec_scheduler_type="multistep",
    epochs_per_stage=(100, 100, 100),
)

opts = make_list(optimisers)
kl_loss = torch.nn.KLDivLoss(reduction='batchmean')
m = nn.Softmax(dim=1)
log_m = nn.LogSoftmax(dim=1)    

#start training
for i_iter in tqdm(range(start_iter, args.num_steps)): 
    loss_seg_cw_value = 0
    loss_seg_sf_value = 0
    loss_fsm_value = 0
    loss_con_value = 0

    for opt in opts:
        opt.zero_grad()

    for sub_i in range(args.iter_size):
        
        # train fog-pass filter
        # and freeze the segmentation network
        model.eval()
        for param in model.parameters():
            param.requires_grad = False
        for param in FogPassFilter1.parameters():
            param.requires_grad = True
        for param in FogPassFilter2.parameters():
            param.requires_grad = True

        _, batch = cwsf_pair_loader_iter_fogpass.__next__()
        sf_image, cw_image, label, size, sf_name, cw_name = batch
        interp = nn.Upsample(size=(size[0][0],size[0][1]), mode='bilinear')
        
        _, batch_rf = rf_loader_iter_fogpass.__next__()
        rf_img,rf_size, rf_name = batch_rf
        img_rf = Variable(rf_img).cuda(args.gpu)
        feature_rf0, feature_rf1, feature_rf2, feature_rf3, feature_rf4, feature_rf5 = model(img_rf) 

        images = Variable(sf_image).cuda(args.gpu)
        feature_sf0,feature_sf1,feature_sf2, feature_sf3,feature_sf4,feature_sf5 = model(images)

        images_cw = Variable(cw_image).cuda(args.gpu)
        feature_cw0, feature_cw1, feature_cw2, feature_cw3, feature_cw4, feature_cw5 = model(images_cw)

        fsm_weights = {'layer0':0.5, 'layer1':0.5}
        sf_features = {'layer0':feature_sf0, 'layer1':feature_sf1}                
        cw_features = {'layer0':feature_cw0, 'layer1':feature_cw1}
        rf_features = {'layer0':feature_rf0, 'layer1':feature_rf1}

        total_fpf_loss = 0

        for idx, layer in enumerate(fsm_weights):
            cw_feature = cw_features[layer]
            sf_feature = sf_features[layer]    
            rf_feature = rf_features[layer]      
            fog_pass_filter_loss = 0 
            
            if idx == 0:
                fogpassfilter = FogPassFilter1
                fogpassfilter_optimizer = FogPassFilter1_optimizer
            elif idx == 1:
                fogpassfilter = FogPassFilter2
                fogpassfilter_optimizer = FogPassFilter2_optimizer

            fogpassfilter.train()  
            fogpassfilter_optimizer.zero_grad()
            
            sf_gram = [0]*args.batch_size
            cw_gram = [0]*args.batch_size
            rf_gram = [0]*args.batch_size 
            vector_sf_gram = [0]*args.batch_size
            vector_cw_gram = [0]*args.batch_size
            vector_rf_gram  = [0]*args.batch_size
            fog_factor_sf = [0]*args.batch_size
            fog_factor_cw = [0]*args.batch_size
            fog_factor_rf = [0]*args.batch_size

            for batch_idx in range(args.batch_size):
                sf_gram[batch_idx] = gram_matrix(sf_feature[batch_idx])
                cw_gram[batch_idx] = gram_matrix(cw_feature[batch_idx])
                rf_gram[batch_idx] = gram_matrix(rf_feature[batch_idx])

                vector_sf_gram[batch_idx] = Variable(sf_gram[batch_idx][torch.triu(torch.ones(sf_gram[batch_idx].size()[0], sf_gram[batch_idx].size()[1])) == 1], requires_grad=True)
                vector_cw_gram[batch_idx] = Variable(cw_gram[batch_idx][torch.triu(torch.ones(cw_gram[batch_idx].size()[0], cw_gram[batch_idx].size()[1])) == 1], requires_grad=True)
                vector_rf_gram[batch_idx] = Variable(rf_gram[batch_idx][torch.triu(torch.ones(rf_gram[batch_idx].size()[0], rf_gram[batch_idx].size()[1])) == 1], requires_grad=True)

                fog_factor_sf[batch_idx] = fogpassfilter(vector_sf_gram[batch_idx])
                fog_factor_cw[batch_idx] = fogpassfilter(vector_cw_gram[batch_idx])
                fog_factor_rf[batch_idx] = fogpassfilter(vector_rf_gram[batch_idx])                                                                                                                                                                                                

            fog_factor_embeddings = torch.cat((torch.unsqueeze(fog_factor_sf[0],0),torch.unsqueeze(fog_factor_cw[0],0),torch.unsqueeze(fog_factor_rf[0],0),
                                                torch.unsqueeze(fog_factor_sf[1],0),torch.unsqueeze(fog_factor_cw[1],0),torch.unsqueeze(fog_factor_rf[1],0),
                                                torch.unsqueeze(fog_factor_sf[2],0),torch.unsqueeze(fog_factor_cw[2],0),torch.unsqueeze(fog_factor_rf[2],0),
                                                torch.unsqueeze(fog_factor_sf[3],0),torch.unsqueeze(fog_factor_cw[3],0),torch.unsqueeze(fog_factor_rf[3],0)),0)

            fog_factor_embeddings_norm = torch.norm(fog_factor_embeddings, p=2, dim=1).detach()
            size_fog_factor = fog_factor_embeddings.size()
            fog_factor_embeddings = fog_factor_embeddings.div(fog_factor_embeddings_norm.expand(size_fog_factor[1],12).t())
            fog_factor_labels = torch.LongTensor([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2])
            fog_pass_filter_loss = fogpassfilter_loss(fog_factor_embeddings,fog_factor_labels)

            total_fpf_loss +=  fog_pass_filter_loss 

        total_fpf_loss.backward(retain_graph=False)

        if args.modeltrain=='train':
            # train segmentation network
            # freeze the fog pass filter

            model.train()
            for param in model.parameters():
                param.requires_grad = True
            for param in FogPassFilter1.parameters():
                param.requires_grad = False
            for param in FogPassFilter2.parameters():
                param.requires_grad = False

            _, batch = cwsf_pair_loader_iter.__next__()
            sf_image, cw_image, label, size, sf_name, cw_name = batch

            interp = nn.Upsample(size=(size[0][0],size[0][1]), mode='bilinear')

            if i_iter % 3 == 0:
                images_sf = Variable(sf_image).cuda(args.gpu)
                feature_sf0,feature_sf1,feature_sf2, feature_sf3,feature_sf4,feature_sf5 = model(images_sf)
                pred_sf5 = interp(feature_sf5)
                loss_seg_sf = loss_calc(pred_sf5, label, args.gpu)
                images_cw = Variable(cw_image).cuda(args.gpu)
                feature_cw0, feature_cw1, feature_cw2, feature_cw3, feature_cw4, feature_cw5 = model(images_cw)
                pred_cw5 = interp(feature_cw5)
                feature_cw5_logsoftmax = log_m(feature_cw5)
                feature_sf5_softmax = m(feature_sf5)
                feature_sf5_logsoftmax = log_m(feature_sf5)
                feature_cw5_softmax = m(feature_cw5)
                loss_con = kl_loss(feature_sf5_logsoftmax, feature_cw5_softmax)
                loss_seg_cw = loss_calc(pred_cw5, label, args.gpu)     
                fsm_weights = {'layer0':0.5, 'layer1':0.5}
                sf_features = {'layer0':feature_sf0, 'layer1':feature_sf1}                
                cw_features = {'layer0':feature_cw0, 'layer1':feature_cw1}

            if i_iter % 3 == 1:
                _, batch_rf = rf_loader_iter.__next__()
                rf_img,rf_size, rf_name = batch_rf
                images_sf = Variable(sf_image).cuda(args.gpu)
                feature_sf0,feature_sf1,feature_sf2, feature_sf3,feature_sf4,feature_sf5 = model(images_sf)
                pred_sf5 = interp(feature_sf5)
                loss_seg_sf = loss_calc(pred_sf5, label, args.gpu)       
                loss_seg_cw = 0   
                loss_con = 0
                img_rf = Variable(rf_img).cuda(args.gpu)
                feature_rf0, feature_rf1, feature_rf2, feature_rf3, feature_rf4, feature_rf5 = model(img_rf)    
                rf_features = {'layer0':feature_rf0, 'layer1':feature_rf1}
                sf_features = {'layer0':feature_sf0, 'layer1':feature_sf1}
                fsm_weights = {'layer0':0.5, 'layer1':0.5}
            
            if i_iter % 3 == 2:
                _, batch_rf = rf_loader_iter.__next__()
                rf_img,rf_size, rf_name = batch_rf
                images_cw = Variable(cw_image).cuda(args.gpu)
                feature_cw0, feature_cw1, feature_cw2, feature_cw3, feature_cw4, feature_cw5 = model(images_cw)
                pred_cw5 = interp(feature_cw5)
                loss_seg_sf = 0
                loss_con = 0
                loss_seg_cw = loss_calc(pred_cw5, label, args.gpu)      
                img_rf = Variable(rf_img).cuda(args.gpu)
                feature_rf0, feature_rf1, feature_rf2, feature_rf3, feature_rf4, feature_rf5 = model(img_rf)                  
                rf_features = {'layer0':feature_rf0, 'layer1':feature_rf1}
                cw_features = {'layer0':feature_cw0, 'layer1':feature_cw1}
                fsm_weights = {'layer0':0.5, 'layer1':0.5}

            loss_fsm = 0
            fog_pass_filter_loss = 0

            for idx, layer in enumerate(fsm_weights):
                # fog pass filter loss between different fog conditions a and b
                if i_iter % 3 == 0:
                    a_feature = cw_features[layer]
                    b_feature = sf_features[layer]    
                if i_iter % 3 == 1:
                    a_feature = rf_features[layer]
                    b_feature = sf_features[layer]
                if i_iter % 3 == 2:
                    a_feature = rf_features[layer]
                    b_feature = cw_features[layer]   

                layer_fsm_loss = 0
                fog_pass_filter_loss = 0   
                na,da,ha,wa = a_feature.size()
                nb,db,hb,wb = b_feature.size()

                if idx == 0:
                    fogpassfilter = FogPassFilter1
                    fogpassfilter_optimizer = FogPassFilter1_optimizer
                elif idx == 1:
                    fogpassfilter = FogPassFilter2
                    fogpassfilter_optimizer = FogPassFilter2_optimizer

                fogpassfilter.eval()

                for batch_idx in range(4):
                    b_gram = gram_matrix(b_feature[batch_idx])
                    a_gram = gram_matrix(a_feature[batch_idx])

                    if i_iter % 3 == 1 or i_iter % 3 == 2:
                        a_gram = a_gram *(hb*wb)/(ha*wa)

                    vector_b_gram = b_gram[torch.triu(torch.ones(b_gram.size()[0], b_gram.size()[1])).requires_grad_() == 1].requires_grad_()
                    vector_a_gram = a_gram[torch.triu(torch.ones(a_gram.size()[0], a_gram.size()[1])).requires_grad_() == 1].requires_grad_()

                    fog_factor_b = fogpassfilter(vector_b_gram)
                    fog_factor_a = fogpassfilter(vector_a_gram)
                    half = int(fog_factor_b.shape[0]/2)
                    
                    layer_fsm_loss += fsm_weights[layer]*torch.mean((fog_factor_b/(hb*wb) - fog_factor_a/(ha*wa))**2)/half/ b_feature.size(0)

                loss_fsm += layer_fsm_loss / 4.

            loss = loss_seg_sf + loss_seg_cw + args.lambda_fsm*loss_fsm + args.lambda_con*loss_con  
            loss = loss / args.iter_size
            loss.backward()

            if loss_seg_cw != 0:
                loss_seg_cw_value += loss_seg_cw.data.cpu().numpy() / args.iter_size
            if loss_seg_sf != 0:
                loss_seg_sf_value += loss_seg_sf.data.cpu().numpy() / args.iter_size
            if loss_fsm != 0:
                loss_fsm_value += loss_fsm.data.cpu().numpy() / args.iter_size
            if loss_con != 0:
                loss_con_value += loss_con.data.cpu().numpy() / args.iter_size   

            for opt in opts:
                opt.step()

        FogPassFilter1_optimizer.step()
        FogPassFilter2_optimizer.step()

    now = datetime.now().strftime('%m-%d-%H-%M')    
    run_name = f'{args.file_name}-{now}'

    if i_iter < 20000:
        save_pred_every = 5000
        if args.modeltrain=='train':
            save_pred_every = 2000
    else:
        save_pred_every = args.save_pred_every

    if i_iter >= args.num_steps_stop - 1:
        print('save model ..')
        torch.save(model.state_dict(), osp.join(args.snapshot_dir, args.file_name + str(args.num_steps_stop) + '.pth'))
        break
    
  
    if args.modeltrain != 'train':
        if i_iter == 5000:
            torch.save({'state_dict':model.state_dict(),
            'fogpass1_state_dict':FogPassFilter1.state_dict(),
            'fogpass2_state_dict':FogPassFilter2.state_dict(),
            'train_iter':i_iter,
            'args':args
            },osp.join(args.snapshot_dir, run_name)+'_fogpassfilter_'+str(i_iter)+'.pth')

    if i_iter % save_pred_every == 0 and i_iter != 0:
        print('taking snapshot ...')
        save_dir = osp.join(f'./result/FIFO_model', args.file_name)
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        torch.save({
            'state_dict':model.state_dict(),
            'fogpass1_state_dict':FogPassFilter1.state_dict(),
            'fogpass2_state_dict':FogPassFilter2.state_dict(),
            'train_iter':i_iter,
            'args':args
        },osp.join(args.snapshot_dir, run_name)+'_FIFO'+str(i_iter)+'.pth')
            

In [ ]:
# Below is the code for our preprocessing layer, do not need to be implement here
# (we add the code in .py file in github)
# just for show

In [ ]:
import cv2
import math
import numpy as np
import statistics
import sys
import os


#Section 3
def getDark(img,kernal_size):
    dc = np.minimum(np.minimum(img[:,:,0],img[:,:,1]),img[:,:,2])
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(kernal_size,kernal_size))
    dark = cv2.erode(dc,kernel)
    return dark

#Section 4.1
def getTransmission(img,A,sz):
    omega = 0.95;
    
    R = img[:,:,0] / A[0,0]
    G = img[:,:,1] / A[0,1]
    B = img[:,:,2] / A[0,2]

    normalized = np.dstack((R,G,B)) 

    transmission = 1 - omega*getDark(normalized,sz)
    return transmission

#modified Section 4.2
#inspired by https://github.com/martiansideofthemoon/blind-dehazing/blob/master/dark_prior/guidedfilter.py
def Guidedfilter(img,transmission,filter_size=60,epsilion=0.0001):
    imageMean= cv2.boxFilter(img,cv2.CV_64F,(filter_size, filter_size))
    transMean = cv2.boxFilter(transmission, cv2.CV_64F,(filter_size, filter_size))
    itMean = cv2.boxFilter(img*transmission,cv2.CV_64F,(filter_size, filter_size))
    itCov = itMean - imageMean*transMean 
    imageVar   = cv2.boxFilter(img*img,cv2.CV_64F,(filter_size, filter_size))- imageMean*imageMean

    a = itCov/(imageVar + epsilion)
    b = transMean - a*imageMean

    aMean = cv2.boxFilter(a,cv2.CV_64F,(filter_size, filter_size))
    bMean = cv2.boxFilter(b,cv2.CV_64F,(filter_size, filter_size))

    res = aMean*img + bMean
    return res

def TransmissionRefine(img,transmission):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray)/255
    t = Guidedfilter(gray,transmission,filter_size=60,epsilion=0.0001)

    return t;


#Section 4.3
def Recover(img,t,A,t_default = 0.1):
    t = cv2.max(t,t_default)
    R = (img[:,:,0]-A[0,0])/t + A[0,0]
    G = (img[:,:,1]-A[0,1])/t + A[0,1]
    B = (img[:,:,2]-A[0,2])/t + A[0,2]
    recovered= np.dstack((R,G,B)) 
    return recovered

#Section4.4
def getA(img,dark):
    pixelNum = img.shape[0] * img.shape[1]
    selectedPixel = int(max(math.floor(pixelNum/1000),1)) #select at least 1 pixel
    darkvector = np.reshape(dark,-1)
    R = np.reshape(img[:,:,0],-1)
    G = np.reshape(img[:,:,1],-1)
    B = np.reshape(img[:,:,2],-1)
    
    idx = darkvector.argsort()
    idx = idx[pixelNum-selectedPixel::]
    A = np.zeros([1,3])
    for i in range(0,selectedPixel):
        A[0,0] = A[0,0] + R[idx[i]]
        A[0,1] = A[0,1] + G[idx[i]]
        A[0,2] = A[0,2] + B[idx[i]]
    
    A = A/selectedPixel
    
    return A

                      
def PreprocessingLayer(img):  #img should be a np array    
    #bilateral filter        
    img = cv2.bilateralFilter(img, 15, 75, 75)
    #dehaze
    dark = getDark(img,15)
    A = getA(img,dark)
    transmission = getTransmission(img,A,15)
    transmissionBetter = TransmissionRefine(img,transmission)
    res = Recover(img,transmissionBetter,A,0.1) + 50
    # add 50 to increase the lightness
    return res


In [ ]:
cd /content/CS639/fifo

/content/CS639/fifo


In [ ]:
#evaluation of original model

import argparse
import numpy as np

IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
DATA_DIRECTORY ='/content/gdrive/MyDrive/CS639/data1'
DATA_CITY_PATH = './dataset/cityscapes_list/clear_lindau.txt'
DATA_DIRECTORY_CITY = '/content/gdrive/MyDrive/CS639/data1/Cityscapes'
DATA_LIST_PATH_EVAL = '/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich/lists_file_names/RGB_testv2_filenames.txt'
DATA_LIST_PATH_EVAL_FD ='./lists_file_names/leftImg8bit_testall_filenames.txt'
DATA_LIST_PATH_EVAL_FDD ='./lists_file_names/leftImg8bit_testdense_filenames.txt' 
DATA_DIR_EVAL = '/content/gdrive/MyDrive/CS639/data1'
DATA_DIR_EVAL_FD = '/content/gdrive/MyDrive/CS639/data1/Foggy_Driving'
NUM_CLASSES = 19 
RESTORE_FROM = '/content/gdrive/MyDrive/CS639/data1/snapshots/FIFO_model/Original_Model_20000.pth'
SNAPSHOT_DIR = f'/content/gdrive/MyDrive/CS639/data1/snapshots/FIFO-val'
GT_DIR_FZ = '/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich'
GT_DIR_FD = '/content/gdrive/MyDrive/CS639/data1/Foggy_Driving'
GT_DIR_CLINDAU = '/content/gdrive/MyDrive/CS639/data1/Cityscapes/gtFine'
SET = 'val'
FILE_NAME = 'Evaluation_Results' 

def get_arguments():
    parser = argparse.ArgumentParser(description="Evlauation")
    parser.add_argument("--data-dir", type=str, default=DATA_DIRECTORY)
    parser.add_argument("--data-city-list", type=str, default = DATA_CITY_PATH)
    parser.add_argument("--data-list-eval-fd", type=str, default=DATA_LIST_PATH_EVAL_FD)      
    parser.add_argument("--data-list-eval-fdd", type=str, default=DATA_LIST_PATH_EVAL_FDD)             
    parser.add_argument("--data-dir-city", type=str, default=DATA_DIRECTORY_CITY)
    parser.add_argument("--data-list-eval", type=str, default=DATA_LIST_PATH_EVAL)
    parser.add_argument("--data-dir-eval", type=str, default=DATA_DIR_EVAL)
    parser.add_argument("--data-dir-eval-fd", type=str, default=DATA_DIR_EVAL_FD)
    parser.add_argument("--num-classes", type=int, default=NUM_CLASSES)
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM)    
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR)
    parser.add_argument("--gpu", type=int, default=0)
    parser.add_argument("--set", type=str, default=SET)
    parser.add_argument("--file-name", type=str, default=FILE_NAME)
    parser.add_argument("--gt-dir-fz", type=str, default=GT_DIR_FZ)
    parser.add_argument("--gt-dir-fd", type=str, default=GT_DIR_FD)
    parser.add_argument("--gt-dir-clindau", type=str, default=GT_DIR_CLINDAU)
    parser.add_argument("--devkit-dir-fz", default='/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich/lists_file_names') 
    parser.add_argument("--devkit-dir-fd", default='./lists_file_names') 
    parser.add_argument("--devkit-dir-clindau", default='./dataset/cityscapes_list')
    return parser.parse_args(args=[])

args = get_arguments()



In [ ]:
import os
import os.path as osp

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data

import argparse
import numpy as np
from packaging import version
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from model.refinenetlw import rf_lw101
from compute_iou import compute_mIoU
from dataset.cityscapes_dataset import cityscapesDataSet
from dataset.Foggy_Zurich_test import foggyzurichDataSet
from dataset.foggy_driving import foggydrivingDataSet

palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)

for i in range(zero_pad):
    palette.append(0)


def colorize_mask(mask):
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)
    return new_mask

args = get_arguments()

#load the trained model
restore = torch.load(args.restore_from)
model = rf_lw101(num_classes=args.num_classes)

model.load_state_dict(restore['state_dict'])
start_iter = 0

#the filepath of the results for different dataset
save_dir_fz = osp.join(f'./result_FZ', args.file_name)
save_dir_fd = osp.join(f'./result_FD', args.file_name)
save_dir_fdd = osp.join(f'./result_FDD', args.file_name)
save_dir_clindau = osp.join(f'./result_Clindau', args.file_name)      

if not os.path.exists(save_dir_fz):
    os.makedirs(save_dir_fz)
if not os.path.exists(save_dir_fd):
    os.makedirs(save_dir_fd)
if not os.path.exists(save_dir_fdd):
    os.makedirs(save_dir_fdd)
if not os.path.exists(save_dir_clindau):
    os.makedirs(save_dir_clindau)

model.eval()
device = torch.device('cuda:0')
model.to(device)

testloader1 = data.DataLoader(foggyzurichDataSet(args.data_dir_eval, args.data_list_eval, crop_size=(1152, 648), mean=IMG_MEAN),
                                batch_size=1, shuffle=False, pin_memory=True)
testloader2 = data.DataLoader(foggyzurichDataSet(args.data_dir_eval, args.data_list_eval, crop_size=(1536, 864), mean=IMG_MEAN),
                                batch_size=1, shuffle=False, pin_memory=True)
testloader3 = data.DataLoader(foggyzurichDataSet(args.data_dir_eval, args.data_list_eval, crop_size=(1920, 1080), mean=IMG_MEAN),
                                batch_size=1, shuffle=False, pin_memory=True)

if version.parse(torch.__version__) >= version.parse('0.4.0'):
    interp_eval = nn.Upsample(size=(1080,1920), mode='bilinear', align_corners=True)
else:
    interp_eval = nn.Upsample(size=(1080,1920), mode='bilinear')

testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)


for index, batch1 in enumerate(testloader1):
    image, label_test, _, name = batch1
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, label_test, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, label_test, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_fz, name))
    output_col.save('%s/%s_color.png' % (save_dir_fz, name[:-4]))

miou_fz = compute_mIoU(args.gt_dir_fz, save_dir_fz, args.devkit_dir_fz, 'FZ')

#test in foggydrivingDataSet
testloader1 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fdd, scale=1),
                                batch_size=1, shuffle=False, pin_memory=True)

testloader2 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fdd, scale=0.8),
                                batch_size=1, shuffle=False, pin_memory=True) 

testloader3 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fdd, scale=0.6),
                                batch_size=1, shuffle=False, pin_memory=True)
testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)

for index, batch in enumerate(testloader1):
    image, size, name = batch
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        interp_eval = nn.Upsample(size=(size[0][0],size[0][1]), mode='bilinear')
        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_fdd, name))
    output_col.save('%s/%s_color.png' % (save_dir_fdd, name[:-4]))
miou_fdd = compute_mIoU(args.gt_dir_fd, save_dir_fdd, args.devkit_dir_fd, 'FDD')


testloader1 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fd, scale=1),
                                batch_size=1, shuffle=False, pin_memory=True) 

testloader2 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fd, scale=0.8),
                                batch_size=1, shuffle=False, pin_memory=True) 

testloader3 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fd, scale=0.6),
                                batch_size=1, shuffle=False, pin_memory=True) 
testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)

for index, batch in enumerate(testloader1):
    image, size, name = batch
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        interp_eval = nn.Upsample(size=(size[0][0],size[0][1]), mode='bilinear')

        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_fd, name))
    output_col.save('%s/%s_color.png' % (save_dir_fd, name[:-4]))
miou_fd = compute_mIoU(args.gt_dir_fd, save_dir_fd, args.devkit_dir_fd, 'FD')


testloader1 = data.DataLoader(cityscapesDataSet(args.data_dir_city, args.data_city_list, crop_size = (2048, 1024), mean=IMG_MEAN, scale=False, mirror=False, set=args.set),
                        batch_size=1, shuffle=False, pin_memory=True)
testloader2 = data.DataLoader(cityscapesDataSet(args.data_dir_city, args.data_city_list, crop_size = (2048*0.8, 1024*0.8), mean=IMG_MEAN, scale=False, mirror=False, set=args.set),
                        batch_size=1, shuffle=False, pin_memory=True)
testloader3 = data.DataLoader(cityscapesDataSet(args.data_dir_city, args.data_city_list, crop_size = (2048*0.6, 1024*0.6), mean=IMG_MEAN, scale=False, mirror=False, set=args.set),
                        batch_size=1, shuffle=False, pin_memory=True)   
testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)

for index, batch in enumerate(testloader1):
    image, size, name = batch
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        interp_eval = nn.Upsample(size=(1024, 2048), mode='bilinear')
        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_clindau, name))
    output_col.save('%s/%s_color.png' % (save_dir_clindau, name.split('.')[0]))

miou_clindau = compute_mIoU(args.gt_dir_clindau, save_dir_clindau, args.devkit_dir_clindau, 'Clindau')



Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.torch/models/101_imagenet.pth.tar


Evaluation on Foggy Zurich
===> mIoU: 42.88
Evaluation on Foggy Driving Dense
===> mIoU: 38.66
Evaluation on Foggy Driving
===> mIoU: 46.9
Evaluation on Cityscapes lindau 40
===> mIoU: 67.04


In [ ]:
cd /content/CS639/fifo

/content/CS639/fifo


In [ ]:
#evaluation of original model  with preprocessing layer

import argparse
import numpy as np

IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
DATA_DIRECTORY ='/content/gdrive/MyDrive/CS639/data1'
DATA_CITY_PATH = './dataset/cityscapes_list/clear_lindau.txt'
DATA_DIRECTORY_CITY = '/content/gdrive/MyDrive/CS639/data1/Cityscapes'
DATA_LIST_PATH_EVAL = '/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich/lists_file_names/RGB_testv2_filenames.txt'
DATA_LIST_PATH_EVAL_FD ='./lists_file_names/leftImg8bit_testall_filenames.txt'
DATA_LIST_PATH_EVAL_FDD ='./lists_file_names/leftImg8bit_testdense_filenames.txt' 
DATA_DIR_EVAL = '/content/gdrive/MyDrive/CS639/data1'
DATA_DIR_EVAL_FD = '/content/gdrive/MyDrive/CS639/data1/Foggy_Driving'
NUM_CLASSES = 19 
RESTORE_FROM = '/content/gdrive/MyDrive/CS639/data1/snapshots/FIFO_model/Original_Model_20000_pre.pth'
SNAPSHOT_DIR = f'/content/gdrive/MyDrive/CS639/data1/snapshots/FIFO-val-pre'
GT_DIR_FZ = '/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich'
GT_DIR_FD = '/content/gdrive/MyDrive/CS639/data1/Foggy_Driving'
GT_DIR_CLINDAU = '/content/gdrive/MyDrive/CS639/data1/Cityscapes/gtFine'
SET = 'val'
FILE_NAME = 'Evaluation_Results_pre' 

def get_arguments():
    parser = argparse.ArgumentParser(description="Evlauation")
    parser.add_argument("--data-dir", type=str, default=DATA_DIRECTORY)
    parser.add_argument("--data-city-list", type=str, default = DATA_CITY_PATH)
    parser.add_argument("--data-list-eval-fd", type=str, default=DATA_LIST_PATH_EVAL_FD)      
    parser.add_argument("--data-list-eval-fdd", type=str, default=DATA_LIST_PATH_EVAL_FDD)             
    parser.add_argument("--data-dir-city", type=str, default=DATA_DIRECTORY_CITY)
    parser.add_argument("--data-list-eval", type=str, default=DATA_LIST_PATH_EVAL)
    parser.add_argument("--data-dir-eval", type=str, default=DATA_DIR_EVAL)
    parser.add_argument("--data-dir-eval-fd", type=str, default=DATA_DIR_EVAL_FD)
    parser.add_argument("--num-classes", type=int, default=NUM_CLASSES)
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM)    
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR)
    parser.add_argument("--gpu", type=int, default=0)
    parser.add_argument("--set", type=str, default=SET)
    parser.add_argument("--file-name", type=str, default=FILE_NAME)
    parser.add_argument("--gt-dir-fz", type=str, default=GT_DIR_FZ)
    parser.add_argument("--gt-dir-fd", type=str, default=GT_DIR_FD)
    parser.add_argument("--gt-dir-clindau", type=str, default=GT_DIR_CLINDAU)
    parser.add_argument("--devkit-dir-fz", default='/content/gdrive/MyDrive/CS639/data1/Foggy_Zurich/lists_file_names') 
    parser.add_argument("--devkit-dir-fd", default='./lists_file_names') 
    parser.add_argument("--devkit-dir-clindau", default='./dataset/cityscapes_list')
    return parser.parse_args(args=[])

args = get_arguments()


In [ ]:
import os
import os.path as osp

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data

import argparse
import numpy as np
from packaging import version
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from model.refinenetlw import rf_lw101
from compute_iou import compute_mIoU
from dataset.cityscapes_dataset_pre import cityscapesDataSet
from dataset.Foggy_Zurich_test_pre import foggyzurichDataSet
from dataset.foggy_driving_pre import foggydrivingDataSet

palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)

for i in range(zero_pad):
    palette.append(0)


def colorize_mask(mask):
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)
    return new_mask

args = get_arguments()

#load the trained model
restore = torch.load(args.restore_from)
model = rf_lw101(num_classes=args.num_classes)

model.load_state_dict(restore['state_dict'])
start_iter = 0

#the filepath of the results for different dataset
save_dir_fz = osp.join(f'./result_FZ', args.file_name)
save_dir_fd = osp.join(f'./result_FD', args.file_name)
save_dir_fdd = osp.join(f'./result_FDD', args.file_name)
save_dir_clindau = osp.join(f'./result_Clindau', args.file_name)      

if not os.path.exists(save_dir_fz):
    os.makedirs(save_dir_fz)
if not os.path.exists(save_dir_fd):
    os.makedirs(save_dir_fd)
if not os.path.exists(save_dir_fdd):
    os.makedirs(save_dir_fdd)
if not os.path.exists(save_dir_clindau):
    os.makedirs(save_dir_clindau)

model.eval()
device = torch.device('cuda:0')
model.to(device)

testloader1 = data.DataLoader(foggyzurichDataSet(args.data_dir_eval, args.data_list_eval, crop_size=(1152, 648), mean=IMG_MEAN),
                                batch_size=1, shuffle=False, pin_memory=True)
testloader2 = data.DataLoader(foggyzurichDataSet(args.data_dir_eval, args.data_list_eval, crop_size=(1536, 864), mean=IMG_MEAN),
                                batch_size=1, shuffle=False, pin_memory=True)
testloader3 = data.DataLoader(foggyzurichDataSet(args.data_dir_eval, args.data_list_eval, crop_size=(1920, 1080), mean=IMG_MEAN),
                                batch_size=1, shuffle=False, pin_memory=True)

if version.parse(torch.__version__) >= version.parse('0.4.0'):
    interp_eval = nn.Upsample(size=(1080,1920), mode='bilinear', align_corners=True)
else:
    interp_eval = nn.Upsample(size=(1080,1920), mode='bilinear')

testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)


for index, batch1 in enumerate(testloader1):
    image, label_test, _, name = batch1
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, label_test, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, label_test, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_fz, name))
    output_col.save('%s/%s_color.png' % (save_dir_fz, name[:-4]))

miou_fz = compute_mIoU(args.gt_dir_fz, save_dir_fz, args.devkit_dir_fz, 'FZ')

#test in foggydrivingDataSet
testloader1 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fdd, scale=1),
                                batch_size=1, shuffle=False, pin_memory=True)

testloader2 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fdd, scale=0.8),
                                batch_size=1, shuffle=False, pin_memory=True) 

testloader3 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fdd, scale=0.6),
                                batch_size=1, shuffle=False, pin_memory=True)
testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)

for index, batch in enumerate(testloader1):
    image, size, name = batch
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        interp_eval = nn.Upsample(size=(size[0][0],size[0][1]), mode='bilinear')
        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_fdd, name))
    output_col.save('%s/%s_color.png' % (save_dir_fdd, name[:-4]))
miou_fdd = compute_mIoU(args.gt_dir_fd, save_dir_fdd, args.devkit_dir_fd, 'FDD')


testloader1 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fd, scale=1),
                                batch_size=1, shuffle=False, pin_memory=True) 

testloader2 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fd, scale=0.8),
                                batch_size=1, shuffle=False, pin_memory=True) 

testloader3 = data.DataLoader(foggydrivingDataSet(args.data_dir_eval_fd, args.data_list_eval_fd, scale=0.6),
                                batch_size=1, shuffle=False, pin_memory=True) 
testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)

for index, batch in enumerate(testloader1):
    image, size, name = batch
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        interp_eval = nn.Upsample(size=(size[0][0],size[0][1]), mode='bilinear')

        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_fd, name))
    output_col.save('%s/%s_color.png' % (save_dir_fd, name[:-4]))
miou_fd = compute_mIoU(args.gt_dir_fd, save_dir_fd, args.devkit_dir_fd, 'FD')


testloader1 = data.DataLoader(cityscapesDataSet(args.data_dir_city, args.data_city_list, crop_size = (2048, 1024), mean=IMG_MEAN, scale=False, mirror=False, set=args.set),
                        batch_size=1, shuffle=False, pin_memory=True)
testloader2 = data.DataLoader(cityscapesDataSet(args.data_dir_city, args.data_city_list, crop_size = (2048*0.8, 1024*0.8), mean=IMG_MEAN, scale=False, mirror=False, set=args.set),
                        batch_size=1, shuffle=False, pin_memory=True)
testloader3 = data.DataLoader(cityscapesDataSet(args.data_dir_city, args.data_city_list, crop_size = (2048*0.6, 1024*0.6), mean=IMG_MEAN, scale=False, mirror=False, set=args.set),
                        batch_size=1, shuffle=False, pin_memory=True)   
testloader_iter2 = enumerate(testloader2)
testloader_iter3 = enumerate(testloader3)

for index, batch in enumerate(testloader1):
    image, size, name = batch
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        interp_eval = nn.Upsample(size=(1024, 2048), mode='bilinear')
        output_1 = interp_eval(output2)

    _, batch2 = testloader_iter2.__next__()
    image, _, name = batch2
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_2 = interp_eval(output2)

    _, batch3 = testloader_iter3.__next__()    
    image, _, name = batch3
    with torch.no_grad():
        output6, output3, output4, output5, output1, output2 = model(Variable(image).cuda(args.gpu))
        output_3 = interp_eval(output2)

    output = torch.cat([output_1,output_2,output_3])
    output = torch.mean(output, dim=0)
    output = output.cpu().numpy()
    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)

    name = name[0].split('/')[-1]
    output.save('%s/%s' % (save_dir_clindau, name))
    output_col.save('%s/%s_color.png' % (save_dir_clindau, name.split('.')[0]))

miou_clindau = compute_mIoU(args.gt_dir_clindau, save_dir_clindau, args.devkit_dir_clindau, 'Clindau')



Evaluation on Foggy Zurich
===> mIoU: 41.42
Evaluation on Foggy Driving Dense
===> mIoU: 40.84
Evaluation on Foggy Driving
===> mIoU: 47.10
Evaluation on Cityscapes lindau 40
===> mIoU: 65.21
